<a href="https://colab.research.google.com/github/Adriano181/folio/blob/main/ejercicio_bi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Puno 1 limpieza de datos

In [2]:
# codigo python

import pandas as pd
import numpy as np

# 1) Cargar el archivo Excel
df_raw = pd.read_excel('assetment-bookmaker.xlsx', sheet_name='in', header=None)

# The sheet is in CSV format within a single column, so we separate it by commas
header_row = df_raw.iloc[0, 0] # Get the header string from the first row
cols = [c.strip() for c in str(header_row).split(',')] # Nombres de columnas
s = df_raw.iloc[1:, 0].astype(str) # Convert all data rows to string for manipulation, excluding the header row
df = s.str.split(',', expand=True) # Separate by commas
df.columns = cols

In [3]:
# 2) Conversión de tipos de datos
df['Periodo'] = pd.to_datetime(df['Periodo'], errors='coerce')

for c in df.columns:
    if c != 'Periodo':
        # Elimina caracteres no numéricos (como símbolos o comas)
        df[c] = pd.to_numeric(df[c].str.replace('[^0-9eE+.-]', '', regex=True), errors='coerce')

In [4]:
# 3) Definimos listas de columnas
count_cols = ['# Clicks', '# Registros IN', '# Primer depÃ³sito IN', '# Personas que apostaron IN']
amount_cols = ['Total depositado IN', 'Total apostado', 'Net revenue IN']

# 4) Imputación (relleno) de valores faltantes
for c in count_cols:
    if c in df.columns:
        df[c] = df[c].fillna(0).astype('Int64')

for c in amount_cols:
    if c in df.columns:
        df[c] = df[c].fillna(0.0)

In [5]:
# 5) Guardar datos limpios para Power BI
df.to_csv('bookmaker_clean_daily.csv', index=False)

Importacion de datos limpios  y  presentacion librerias

In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

# Cargar datos limpios
df = pd.read_csv('bookmaker_clean_daily.csv', parse_dates=['Periodo'])

# Aseguramos que está ordenado por fecha
df = df.sort_values('Periodo')

Agregar datos por mes
Para proyecciones financieras, es más estable trabajar con datos mensuales que diarios.

In [7]:
df_month = df.set_index('Periodo').resample('M').agg({
    'Net revenue IN': 'sum'
}).rename(columns={'Net revenue IN':'net_revenue'}).reset_index()

# Creamos índice temporal (meses desde el inicio)
df_month['month_index'] = (df_month['Periodo'] - df_month['Periodo'].min()).dt.days // 30

/tmp/ipython-input-4220529799.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_month = df.set_index('Periodo').resample('M').agg({


Preparar datos para entrenamiento y prueba
Vamos a separar los últimos 6 meses como test para validar el modelo.

```
# This is formatted as code
```



In [8]:
X = df_month[['month_index']]
y = df_month['net_revenue']

test_size = 6
X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]


Entrenar modelo de regresión lineal
Se ajusta una línea recta a la tendencia mensual.

In [9]:
model = LinearRegression()
model.fit(X_train, y_train)

# Predicción sobre datos de test
y_pred_test = model.predict(X_test)

# Calcular error MAPE
mape_test = mean_absolute_percentage_error(y_test.replace(0, np.nan).fillna(1e-8), y_pred_test)
print(f"MAPE en test: {mape_test:.2%}")

MAPE en test: 33.21%


 Proyectar 24 meses hacia adelante
Creamos nuevos índices de mes y predecimos.

In [10]:
  # Último índice de mes en el dataset
  last_index = df_month['month_index'].max()

  # Generar próximos 24 meses
  future_indices = np.arange(last_index+1, last_index+1+24).reshape(-1,1)
  future_preds = model.predict(future_indices)

  # Crear DataFrame de proyección
  future_dates = pd.date_range(start=df_month['Periodo'].max() + pd.offsets.MonthBegin(1), periods=24, freq='M')
  proj = pd.DataFrame({
      'Periodo': future_dates,
      'net_revenue_pred': future_preds
  })

  print(proj.head())

     Periodo  net_revenue_pred
0 2023-10-31      40215.045557
1 2023-11-30      43135.457748
2 2023-12-31      46055.869939
3 2024-01-31      48976.282131
4 2024-02-29      51896.694322


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/tmp/ipython-input-1414476698.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start=df_month['Periodo'].max() + pd.offsets.MonthBegin(1), periods=24, freq='M')


In [11]:
proj.to_csv('bookmaker_net_revenue_projection_24m.csv', index=False)

In [14]:
# alternativa de codigo conjunto
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from datetime import timedelta

# 1) Cargar
df_raw = pd.read_excel('assetment-bookmaker.xlsx', sheet_name='in', header=None)
# la hoja tiene una sola columna con valores CSV -> separar por coma
header_row = df_raw.iloc[0, 0] # Get the header string from the first row
cols = [c.strip() for c in str(header_row).split(',')] # Nombres de columnas
s = df_raw.iloc[1:, 0].astype(str) # Convert all data rows to string for manipulation, excluding the header row
df = s.str.split(',', expand=True) # Separate by commas
df.columns = cols

# Fix encoding issue in column names
df.columns = df.columns.str.replace('Ã³', 'ó')

# 2) Tipos
df['Periodo'] = pd.to_datetime(df['Periodo'], errors='coerce')
for c in df.columns:
    if c != 'Periodo':
        df[c] = pd.to_numeric(df[c].str.replace('[^0-9eE+.-]', '', regex=True), errors='coerce')

# 3) Estrategia de imputación recomendada (explicada abajo)
# - Para contadores (# Clicks, # Registros, # Primer depósito, # Personas...) rellenar con 0 si significan "no hubo"
# - Para montos (Total depositado, Total apostado, Net revenue) rellenar con 0 si se entiende que no hubo operación,
#   o con la mediana mensual si faltan por errores de ETL. Aquí dejamos opción 1: 0 (más conservadora).
count_cols = ['# Clicks', '# Registros IN', '# Primer depósito IN', '# Personas que apostaron IN']
amount_cols = ['Total depositado IN', 'Total apostado', 'Net revenue IN']

for c in count_cols:
    if c in df.columns:
        df[c] = df[c].fillna(0).astype('Int64')

for c in amount_cols:
    if c in df.columns:
        # alternativa: df[c] = df.groupby(df['Periodo'].dt.to_period('M'))[c].transform(lambda x: x.fillna(x.median()))
        df[c] = df[c].fillna(0.0)

# 4) Agregación mensual
df_month = df.set_index('Periodo').resample('M').agg({
    'Net revenue IN': 'sum',
    'Total apostado': 'sum',
    '# Personas que apostaron IN': 'sum'
}).rename(columns={'Net revenue IN':'net_revenue', 'Total apostado':'total_apostado', '# Personas que apostaron IN':'num_players'})

df_month = df_month.reset_index()
df_month['month_index'] = (df_month['Periodo'] - df_month['Periodo'].min()).dt.days // 30  # índice temporal simple

# 5) Modelo de proyección simple: regresión lineal sobre index (explicable y rápido)
X = df_month[['month_index']]
y = df_month['net_revenue'].fillna(0)

# Train/test (últimos 6 meses test)
test_size = 6
X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]

model = LinearRegression()
model.fit(X_train, y_train)
y_pred_test = model.predict(X_test)

mape_test = mean_absolute_percentage_error(y_test.replace(0, np.nan).fillna(1e-8), y_pred_test)
print('MAPE test (aprox):', mape_test)

# Proyección 24 meses hacia adelante
last_index = X['month_index'].max()
future_indices = np.arange(last_index+1, last_index+1+24).reshape(-1,1)
future_preds = model.predict(future_indices)

future_dates = pd.date_range(start=df_month['Periodo'].max() + pd.offsets.MonthBegin(1), periods=24, freq='M')
proj = pd.DataFrame({'Periodo': future_dates, 'month_index': future_indices.flatten(), 'net_revenue_pred': future_preds})

# 6) Guardar datasets para Power BI
df.to_csv('bookmaker_clean_daily.csv', index=False)
df_month.to_csv('bookmaker_monthly.csv', index=False)
proj.to_csv('bookmaker_net_revenue_projection_24m.csv', index=False)

# 7) Resultado: df_month (histórico) y proj (proyección)
print(df_month.tail())
print(proj.head())

MAPE test (aprox): 0.3320904248964403
      Periodo   net_revenue  total_apostado  num_players  month_index
9  2023-05-31  44567.714103    1.850204e+06         5572            9
10 2023-06-30  26305.867537    6.242841e+05         4907           10
11 2023-07-31  47186.412694    1.557901e+06        21644           11
12 2023-08-31  98242.521515    2.383450e+06        30080           12
13 2023-09-30  52025.806147    1.354040e+06        16955           13
     Periodo  month_index  net_revenue_pred
0 2023-10-31           14      40215.045557
1 2023-11-30           15      43135.457748
2 2023-12-31           16      46055.869939
3 2024-01-31           17      48976.282131
4 2024-02-29           18      51896.694322


/tmp/ipython-input-3477044185.py:40: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_month = df.set_index('Periodo').resample('M').agg({
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/tmp/ipython-input-3477044185.py:70: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start=df_month['Periodo'].max() + pd.offsets.MonthBegin(1), periods=24, freq='M')
